## Importing the Necessary Libraries

In [75]:
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from rich import print

## Link creation

In [51]:
Job = 'Dental Jobs'
search_params = [
    'Boston, MA. 100miles',
    'Houston TX 100 miles',
    'Greensboro, NC. 50 miles',
    'High point NC 50 miles', 
    'Wintson-Salem NC  50 miles',
    'Los Angeles, CA 100 miles',
    'Cleveland OH 100miles',
]

full_search_prompt = [(Job+' '+ search_param) for search_param in search_params]

In [52]:
def build_google_search_url(query):
    """
    Constructs a Google search URL based on the provided search query.

    Args:
      query: The search query string.

    Returns:
      A string containing the formatted Google search URL.
    """
    # Encode special characters in the query for safe URL inclusion
    encoded_query = quote(query)
    # Base URL for Google Search
    google_search_url = "https://www.google.com/search?q="

    # Combine the base URL and encoded query
    return google_search_url + encoded_query

In [53]:
urls = [build_google_search_url(prompt) for prompt in full_search_prompt]

## Get the Webdrivers Ready to work

In [54]:
# Set chrome to headless
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Add the driver to be able to load the pages
browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = chrome_options)

## Get the Google search Page and the html of the page

In [69]:
def extract_job_details(job_div):
    """
    Extracts job title, location (if available), and link from a job posting div.

    Args:
      job_div: A string containing the HTML content of the job posting div.

    Returns:
      A dictionary containing extracted job details, or None if no details are found.
    """

    # Find the job title element (assuming h3 with specific class)
    job_title_element = job_div.find('h3', class_='LC20lb MBeuO DKV0Md')

    if job_title_element:
        # Extract job title
        job_title = job_title_element.text.strip()

        # Find location information (might be within the same element or separate)
        location_element = job_title_element
        location_text = location_element.text.strip()
        location_parts = [part.strip() for part in location_text.split(',') if part.strip()]  # Split and clean location

        # Find the job link
        job_link_element = job_div.find('a')
        if job_link_element:
            job_link = job_link_element['href']
        else:
            job_link = None

        # Create dictionary with extracted details
        job_details = {
            'Title': job_title,
            'Location': ', '.join(location_parts) if location_parts else None,
            'Job Link': job_link
        }
        return job_details
    else:
        return None  # No details found

In [73]:
all_jobs = []
for url in urls:
    browser.get(url)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    content = browser.page_source
    soup = BeautifulSoup(content, 'html.parser')

    link_cards = soup.find_all('div', class_='g Ww4FFb vt6azd tF2Cxc asEBEc')

    all_job = [extract_job_details(link_cards[i]) for i in range(len(link_cards))]
    all_jobs.extend(all_job)

In [71]:
# Convert list of records into a DataFrame
print('Converting to Dataframe')
df = pd.DataFrame(all_jobs)
# Save DataFrame to a CSV file
df.to_csv('google_jobs_data.csv', index=False)

print("Data saved to indeeds_job_data.csv")

Converting to Dataframe

Data saved to indeeds_job_data.csv

In [72]:
dfx = pd.read_csv('google_jobs_data.csv')
dfx

,Title,Location,Job Link
0,"169 General Dentist jobs in Boston, Massachuse...","169 General Dentist jobs in Boston, Massachuse...",https://www.linkedin.com/jobs/general-dentist-...
1,"68 Dentist Jobs in Boston, MA","68 Dentist Jobs in Boston, MA",https://www.ziprecruiter.com/Jobs/Dentist/-in-...
2,"276 Dentist jobs in Boston, Massachusetts, Uni...","276 Dentist jobs in Boston, Massachusetts, Uni...",https://www.linkedin.com/jobs/dentist-jobs-bos...
3,"Dentist Boston Jobs, Employment in Boston, MA","Dentist Boston Jobs, Employment in Boston, MA",https://www.indeed.com/q-dentist-boston-l-bost...
4,"$18-$27/hr Dental Assistant Jobs in Boston, MA...","$18-$27/hr Dental Assistant Jobs in Boston, MA...",https://www.ziprecruiter.com/Jobs/Dental-Assis...
...,...,...,...
65,"Dentist jobs in Cleveland, OH","Dentist jobs in Cleveland, OH","https://www.indeed.com/q-dentist-l-cleveland,-..."
66,"624 Dentist jobs in Cleveland, Ohio, United St...","624 Dentist jobs in Cleveland, Ohio, United St...",https://www.linkedin.com/jobs/dentist-jobs-cle...
67,Jobs | Heartland Dental and Supported Offices ...,Jobs | Heartland Dental and Supported Offices ...,https://jobs.heartland.com/jobs/
68,"14000+ Dental jobs in Cleveland, Ohio, United ...","14000+ Dental jobs in Cleveland, Ohio, United ...",https://www.linkedin.com/jobs/dental-jobs-clev...
